In [1]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 2.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 67.3 MB/s 
     |████████████████████████████████| 3.3 MB 64.8 MB/s 
     |████████████████████████████████| 636 kB 71.1 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 50.1 MB/s 
     |████████████████████████████████| 119 kB 74.4 MB/s 


In [2]:
cd '/content/drive/MyDrive/CAMemBERT2'

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer_f import MultiTaskModelTrainer
from trainer_utils import (training_args,training_kwargs,get_dataset,
                           freeze_layers,normalise_scores,create_params_grid)

In [4]:
for pretrained_model_name in ['distilroberta-base']:
    model_type = 'roberta' if 'roberta' in pretrained_model_name else 'bert'
    model_n_layers = 6 if 'roberta' in pretrained_model_name else 12
    params_grid = create_params_grid({'tasks':[['aes','ged','ner']],
                                      'lr':[4e-5],
                                      'shared_encoder_n_layers':[0,1,2],
                                      'frozen_layers':[None],
                                      'output_layer':[2,4,-1]}
                                      )

    prev_task = None
    for i,params in enumerate(params_grid):
        print(params)
        if params['tasks']!=prev_task:
            dataset_obj,dataset_dict = get_dataset(params['tasks'],pretrained_model=pretrained_model_name,max_length=512)
            dataset_dict = dataset_dict.map(normalise_scores)

        trainer_kwargs,model_kwargs = training_kwargs(
            tasks=params['tasks'],
            class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
            early_stopping_patience=2,
            shared_encoder_n_layers=params['shared_encoder_n_layers'],
            output_layer = {'aes':-1,
                            'ged':params['output_layer'],
                            'ner':params['output_layer']},
            early_stopping_metric='los',
            normalised_values=[40,0],
            frozen_layers=params['frozen_layers'],
            save_results=True)

        args = training_args(lr=params['lr'],epochs=8)
        model = MultiTaskModel(kwargs_dict = model_kwargs,pretrained_model=pretrained_model_name)
        model = freeze_layers(model,frozen_layers=params['frozen_layers'],model_type=model_type)
        dev_set = 'dev' if params['tasks']!='ner' else 'validation'
        trainer = MultiTaskModelTrainer(
            model,
            args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict[dev_set],
        )
        trainer.kwargs_dict = trainer_kwargs
        trainer.test_dataset = dataset_dict['test']
        trainer.train()
        prev_task=params['tasks']

{'frozen_layers': None, 'lr': 4e-05, 'output_layer': 2, 'shared_encoder_n_layers': 0, 'tasks': ['aes', 'ged', 'ner']}


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Step,Training Loss
500,0.384500


aes_weight_coef : 1.0
ged_weight_coef : 1.0
ner_weight_coef : 1.0

losses {'aes': 0.0037696647652706436, 'ged': 0.11997245633324911, 'ner': 0.04723094620330389, 'total_loss': 0.17097306730182366}

aes_metrics {'rmse_aes': 5.381294763654314, 'pearson_aes': 0.5512287341264823, 'spearman_aes': 0.5548952201795083, 'kappa_aes': 0.3752124460713818}

ged               precision    recall  f1-score   support

           0       0.92      0.91      0.91     25174
           1       0.46      0.52      0.49      3917

    accuracy                           0.85     29091
   macro avg       0.69      0.71      0.70     29091
weighted avg       0.86      0.85      0.86     29091

ged_metrics {'accuracy_ged': 0.8542160805747482, 'f1_score_macro_ged': 0.7025750520363769, 'f1_score_weighted_ged': 0.8577567073242721, 'f_0_5_ged': 0.5079720976581964}

ner               precision    recall  f1-score   support

           0  0.995982810 0.997310508 0.996646217     42759
           1  0.979902227 0.979370

Step,Training Loss
500,0.384500
1000,0.241900
1500,0.203500


aes_weight_coef : 1.0
ged_weight_coef : 1.0
ner_weight_coef : 1.0

losses {'aes': 0.003126571501800141, 'ged': 0.09568054870117543, 'ner': 0.02922817625560213, 'total_loss': 0.1280352964585777}

aes_metrics {'rmse_aes': 4.873397172404482, 'pearson_aes': 0.6400987161843409, 'spearman_aes': 0.6576192098900012, 'kappa_aes': 0.5645848724214354}

ged               precision    recall  f1-score   support

           0       0.96      0.82      0.88     25174
           1       0.40      0.77      0.53      3917

    accuracy                           0.81     29091
   macro avg       0.68      0.80      0.71     29091
weighted avg       0.88      0.81      0.84     29091

ged_metrics {'accuracy_ged': 0.8140662060431061, 'f1_score_macro_ged': 0.7057519140399675, 'f1_score_weighted_ged': 0.8362015970490734, 'f_0_5_ged': 0.6502242152466369}

ner               precision    recall  f1-score   support

           0  0.997637758 0.997567764 0.997602760     42759
           1  0.983188720 0.98425624

KeyboardInterrupt: ignored